# Gensim and Sklearn

## Preprocess texts

In [1]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
from IPython.core.display import HTML 
morph = MorphAnalyzer()

In [2]:
stops = set(stopwords.words('russian'))

Normalizing texts

In [3]:
class TextNormalizer():
    
    # Class for fast text-normalization
        
    def opt_normalize(self, texts, top=None) -> list:
        
        texts = self.tokenize_texts(texts)
        
        uniq = Counter()
        [uniq.update(text) for text in texts]

        norm_uniq = { # build cache
            word:morph.parse(
                word
            )[0].normal_form for word, _ in uniq.most_common(top)}

        norm_texts = self.normalize_texts(texts, norm_uniq)
        
        return norm_texts
    
    def tokenize_texts(self, texts):
        """ a text tokenizer for a list of texts """
        return ([self.tokenize(text.lower()) for text in texts])


    def tokenize(self, text) -> list:
        """ a single text tokenizer """
        punct = string.punctuation + '«»—…–“”#'
        words = [word.strip(punct) for word in text.split()]
        words = [word for word in words if word]

        return words
    
    def normalize_texts(self, texts, norm_uniq) -> list:
        """ returns lemmas and removes stopwords and punctuation """
        norm_texts = []
        for text in texts:
            # lemmas from cashe
            norm_words = [norm_uniq.get(word) for word in text]
            #remove punctuation
            
            norm_words = [word for word in norm_words if (
                word) and (word not in stops)]
            norm_texts.append(norm_words)

        return norm_texts

In [4]:
texts = open('data/wiki_data.txt').read().splitlines()[:10000]

In [5]:
%%time
texts = TextNormalizer().opt_normalize(texts, 30000)

CPU times: user 2.27 s, sys: 74.4 ms, total: 2.35 s
Wall time: 2.35 s


In [7]:
texts[0][:5]

['нижегородский', 'сельский', 'посёлок', 'район', 'нижегородский']

build ngrams

In [8]:
# play around with threshold
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.45)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

# Build dictionary for models

In [9]:
dictionary = gensim.corpora.Dictionary(texts)
dictionary.filter_extremes(no_above=0.2, no_below=15)
dictionary.compactify()
print(dictionary)

Dictionary(6846 unique tokens: ['1', '1,2', '1,5', '12', '14']...)


transform to corpora

In [10]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [14]:
lda1 = gensim.models.LdaMulticore(corpus,
                                 100,
                                 id2word=dictionary,
                                 passes=10)

lda2 = gensim.models.LdaMulticore(corpus,
                                 50,
                                 id2word=dictionary,
                                 passes=15)

lda3 = gensim.models.LdaMulticore(corpus,
                                 150,
                                 id2word=dictionary,
                                 passes=5)

check topics

In [54]:
def get_topics(model):
    topics = []
    for id, topic in model.show_topics(num_topics=100, formatted=False):
        topic = [word for word, _ in topic]
        topic.insert(0, str(id))
        topics.append(topic)
    
    return topics

In [55]:
topics1 = get_topics(lda1)
topic2 = get_topics(lda2)
topics3 = get_topics(lda3)

In [60]:
for topic in topics_sm:
    print(" ".join(topic[0:7]))

0 стадион соревнование являться высокий люк проводиться
1 итальянский норвежский карл германия италия немецкий
2 хутор ростовский памятник сельский области.############входить объект
3 университет факультет право медицинский государственный наука
4 катание использовать специалист навык воин игра
5 группа альбом песня the выпустить of
6 олимпийский игра летний участие медаль принимать
7 израиль ван еврейский израильский сторона католический
8 змея полевой пуля кладбище город ружьё
9 чемпионат команда мир сезон кубок сборная
10 турнир финал проиграть раунд чемпионат второй
11 город население центр житель век округ
12 игра компания мочь версия уровень 2010
13 театр институт университет работа г русский
14 улица дом здание название город завод
15 список фамилия азербайджан дата азербайджанский орден
16 храм церковь икона день праздник православный
17 племя ирландия британский канада ирландский лорд
18 поселение сельский воронежский образование муниципальный центр
19 русский александр орден